## Cleaning up Atlas data
**Function**      : Preprocess netCDF files and restructure the dataset<br>
**Author          : Team BETA**<br>
**First Built**   : 2021.08.11<br>
**Last Update     : 2021.08.11**<br>
**Library**       : os, numpy, netcdf4, xarray<br>
**Description**   : In this notebook serves to clean up Atlas data which is given in netcdf format and aggregate the data into a single file.<br>
**Return Values   : .nc files**<br>
**Note**          : All the data is saved to netCDF4 format. Note that data from different models may vary concerning the resolution and coordinates.<br>

In [2]:
import os
import xarray as xr

### Path
Specify the path to the dataset and the place to save the outputs. <br>

In [3]:
# please specify data path
datapath = '/mnt/d/NLeSC/BETA/EUCP/Atlas'
# please specify output path
output_path = '../assets/sample_figures/test'
os.makedirs(output_path, exist_ok = True)

### Extract data
Extract weather/climate data from given netCDF files.

In [5]:
# CNRS
# model unknown
dataset = xr.open_dataset(os.path.join(datapath,'Rita_Lukas_Atlas',
                          'eur_CMIP5_pr_2041-2060_vs_1995-2014_10_JJA.nc'))
dataset

<xarray.Dataset>
Dimensions:        (lon: 20, lat: 17)
Coordinates:
  * lon            (lon) float64 -8.75 -6.25 -3.75 -1.25 ... 33.75 36.25 38.75
  * lat            (lat) float64 31.25 33.75 36.25 38.75 ... 66.25 68.75 71.25
Data variables:
    pr_weighted    (lat, lon) float64 ...
    pr_unweighted  (lat, lon) float64 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.7.1 (http://mpimet.mpg.d...
    Conventions:  CF-1.6
    history:      Sat Jun 19 10:31:00 2021: cdo merge pr_CMIP5_p10_w_JJA.nc p...
    NCO:          netCDF Operators version 4.7.4 (http://nco.sf.net)
    CDO:          Climate Data Operators version 1.9.7.1 (http://mpimet.mpg.d...